## План работы

[1. Загрузка и подготовка данных](#1.)<br>
[2. Обучение и проверка модели](#2.)<br>
[3. Подготовка к расчёту прибыли](#3.)<br>
[4. Расчёт прибыли и рисков](#4.)<br>

# 1. Загрузка и подготовка данных <a id = "1."> </a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statistics
import math

warnings.filterwarnings('ignore') 
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
%matplotlib inline

geo_data_0 = pd.read_csv(r'E:\Документы прочие\IT\Яндекс практикум\Project\7 Машинное обучение в бизнесе\geo_data_0.csv')
geo_data_1 = pd.read_csv(r'E:\Документы прочие\IT\Яндекс практикум\Project\7 Машинное обучение в бизнесе\geo_data_1.csv')
geo_data_2 = pd.read_csv(r'E:\Документы прочие\IT\Яндекс практикум\Project\7 Машинное обучение в бизнесе\geo_data_2.csv')

def data_info(data):
    '''
    Функция принимает в качестве аргумента датасет.
    Функция выводит информацию о датасете и первые 10 строк датасета, а также количество дубликатов.
    '''
    data.info()
    display(data.head(10))
    print('Количество дубликатов в данных:', data.duplicated().sum())
    
def ml_for_lr(data_train, data_valid):
    '''
    Функция принимает в качестве аргументов: обучающую и валидационную выборку.
    Функция обучает модель линейной регрессии на обучающей выборке и расчитывает метрики по валидационной: RMSE и R2.
    В качестве результата функция возвращает предсказанные и правильные значения.
    '''
    # разобьем данные обучающей выборки
    features_train = data_train.drop(['product'], axis = 1)
    target_train = data_train['product']
    # разобьем данные тестовой выборки
    features_valid = data_valid.drop(['product'], axis = 1)
    target_valid = data_valid['product']
    # обучим модель
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    predictions = pd.Series(predictions)
    mse = mean_squared_error(target_valid, predictions)
    rmse = mse**0.5
    mean = target_valid.mean()
    print('Cредний запас сырья: {:.2f}'.format(mean))
    print('RMSE(квадратный корень из средней квадратичной ошибки): {:.2f}'. format(rmse))
    print('R2(коэффициент детерминации): {:.2f}'. format(r2_score(target_valid, predictions)))
    return predictions, target_valid

def test_split(data):
    '''
    Функция принимает в качестве аргумента датасет.
    Функция разделяет выборку на обучающую и валидационную и делает reset индексов.
    '''
    data_train, data_valid = train_test_split(data, test_size = .25, random_state = 12345)
    data_train = data_train.reset_index(drop = True)
    data_valid = data_valid.reset_index(drop = True)
    return data_train, data_valid

def bootstrap(target, probabilities):
    '''
    Функция принимает в качестве аргементов: ответы, предсказания.
    Фукнция вычисляет распределение прибыли с помощью техники bootstrap.
    '''
    # чтобы случайный элемент был всегда разным создадим переменную state
    state = np.random.RandomState(12345)
    # применим технику бутстреп для расчета доверительного интервала
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=SEARCH, replace=True, random_state=state)
        probs_subsample = probabilities[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample, 200))
    values = pd.Series(values)
    #определим среднее значение
    mean = values.mean()
    # определим 95%-й доверительный интервал
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    # определим риск убытков
    loss_count = values.where(values < 0).count()
    risk_of_loss = loss_count / values.count()
    print('Риск убытков: {:.2%}'. format(risk_of_loss))
    print("Средняя прибыль: {:.2f}". format(mean/10**9))
    print("2.5%-квантиль: {:.2f}". format(lower/10**9))
    print("97.5%-квантиль: {:.2f}". format(upper/10**9))

Для анализа мы будем использовать датасеты: geo_data_0, geo_data_1, geo_data_2. Они содержат данные геологоразведки трёх регионов.

In [2]:
data_info(geo_data_0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100000 non-null  int64  
 1   id          100000 non-null  object 
 2   f0          100000 non-null  float64
 3   f1          100000 non-null  float64
 4   f2          100000 non-null  float64
 5   product     100000 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 4.6+ MB


,Unnamed: 0,id,f0,f1,f2,product
0,0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,2,409Wp,1.022732,0.151990,1.419926,85.265647
3,3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,9,OLuZU,2.173381,0.563698,9.441852,127.910945


Количество дубликатов в данных: 0


In [3]:
data_info(geo_data_1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100000 non-null  int64  
 1   id          100000 non-null  object 
 2   f0          100000 non-null  float64
 3   f1          100000 non-null  float64
 4   f2          100000 non-null  float64
 5   product     100000 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 4.6+ MB


,Unnamed: 0,id,f0,f1,f2,product
0,0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


Количество дубликатов в данных: 0


In [4]:
data_info(geo_data_2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100000 non-null  int64  
 1   id          100000 non-null  object 
 2   f0          100000 non-null  float64
 3   f1          100000 non-null  float64
 4   f2          100000 non-null  float64
 5   product     100000 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 4.6+ MB


,Unnamed: 0,id,f0,f1,f2,product
0,0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,9,OilcN,-1.214452,-0.439314,5.922514,52.954532


Количество дубликатов в данных: 0


Описание данных в датасетах:
##### Признаки
- id — уникальный идентификатор месторождения
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы)

##### Целевой признак
- product — объём запасов в месторождении (тыс. баррелей).

Датасеты содержат колонку id с уникальным идентификатором месторождения. Она несет информативный характер и не оказывает влияния на целевой признак, поэтому мы не будем проводить ее кодировку и удалим в каждом датасете.

In [5]:
geo_data_0 = geo_data_0.drop(['id'], axis = 1)
geo_data_1 = geo_data_1.drop(['id'], axis = 1)
geo_data_2 = geo_data_2.drop(['id'], axis = 1)

### Вывод

Была изучена общая информация о полученных данных. Все 3 таблицы размером в 100000 строк состоит из 5 колонок и содержат данные 2-х типов: float64 и object. Данные не содержат пропусков или дубликатов. В данных присутствуют только количественные признаки.

# 2. Обучение и проверка модели <a id = "2."> </a>

#### Веделим из данных валидационную выборку

Для дальнейшей валидации моделей мы выделим из данных валидационные выборки в размере 25% от общего объема.

In [6]:
# выделим валидационную выборку из исходных данных для региона 0
geo_data_0_train, geo_data_0_valid = test_split(geo_data_0)
# выделим валидационную выборку из исходных данных для региона 1
geo_data_1_train, geo_data_1_valid = test_split(geo_data_1)
# выделим валидационную выборку из исходных данных для региона 2
geo_data_2_train, geo_data_2_valid = test_split(geo_data_2)

#### Обучим модели и сделаем предсказания на валидационной выборке

Для обучения модели подходит только линейная регрессия. Остальные модели недостаточно предсказуемые. Обучим модель и сделаем предсказания на валидационной выборке для каждого региона. Запишем результаты предсказаний и правильные ответы в переменные и проанализируем метрики.

#### Регион - 1

In [7]:
predictions_0, target_valid_0 = ml_for_lr(geo_data_0_train, geo_data_0_valid)

Cредний запас сырья: 92.08
RMSE(квадратный корень из средней квадратичной ошибки): 37.58
R2(коэффициент детерминации): 0.28


Не смотря на то, что занчение метрики модели R2 существенно ниже 1, все же она предсказывает объем лучше среднего. RMSE всего лишь составляет 2.4 от среднего запаса сырья по региону. Столь высокая ошибка влечет за собой как правило увеличение рисков. Однако высокий риск от части компенсируется высоким средним запасом сырья на одной скважине, большим, чем у второго региона на 23 тыс. баррелей.

#### Регион - 2

In [8]:
predictions_1, target_valid_1 = ml_for_lr(geo_data_1_train, geo_data_1_valid)

Cредний запас сырья: 68.72
RMSE(квадратный корень из средней квадратичной ошибки): 0.89
R2(коэффициент детерминации): 1.00


Значение метрики R2 равно единице только в одном случае, если MSE нулевое. Как мы видим RMSE - корень из MSE близок к нулю. Можно сделать вывод о том, что полученная модель по данному региону предсказывает объем нефти на скважине идеально. Средний запас сырья в данном регионе составляет 69 тыс. баррелей на одну скважину.

#### Регион - 3

In [9]:
predictions_2, target_valid_2 = ml_for_lr(geo_data_2_train, geo_data_2_valid)

Cредний запас сырья: 94.88
RMSE(квадратный корень из средней квадратичной ошибки): 40.03
R2(коэффициент детерминации): 0.21


В моделе по последнему региону ситуция аналогичная той, которую мы наблюдали в модели первого региона. Модель предсказывает лучше среднего значения, однако ей далеко до идельной. Не смотря на самую высокую среди 3-х моделей ошибку в 40 тыс. баррелей на одну скважину, регион обладает самым высоким средним запасом сырья на скважину.

### Вывод

Мы обучили модели линейной регрессии для 3-х регионов. Регион - 2 оказался худшим по среднему запасу сырья на скважину, но модель построенная для данного региона идально предсказывает объем сырья на скажине. Регионы 1 и 3 являются самыми рисковыми, так как их модели обладают большими RMSE, но также и самими богатыми, так как средний запас сырья на скажине превышает значение в 90 баррелей.

# 3. Подготовка к расчёту прибыли <a id = "3."> </a>

При разведке региона проводится исследование 500 точек.

In [10]:
SEARCH = 500

Бюджет на разработку месторождений — 10 млрд рублей.

In [11]:
BUDGET_FOR_REGION = 10**10

Cтоимость бурения одной скважины — 50 млн рублей.

In [12]:
COST_FOR_WELL = 50*(10**6)

Один баррель сырья приносит 4500 рублей прибыли.

In [13]:
BAR_PRICE = 4500

Имея определенный бюджет, мы будем стремиться потратить его весь, так как это повысит наши шансы на получение большей прибыли. Наш бюджет имеет 1 укрупненную статью расходов - стоимость бурения скважины. Вычислим какое максимальное количество скважин мы можем пробурить на имеющийся бюджет.

In [14]:
max_well = BUDGET_FOR_REGION / COST_FOR_WELL
print('Максимально возможное количество разработанных скажин в регионе: {:.0f}'. format(max_well))

Максимально возможное количество разработанных скажин в регионе: 200


Посчитаем необходимый минимальный объём сырья с одной скважины, который окупит эту скважину.

In [15]:
min_product_well = COST_FOR_WELL/BAR_PRICE/1000
print('Минимальный объем для окупаемости скважины равен {:.2f} тыс. баррелей'.format(min_product_well))

Минимальный объем для окупаемости скважины равен 11.11 тыс. баррелей


Для того, чтобы окупить бурение 1 скважины необходимо с нее добыть не менее 11.11 тыс. баррелей. Посчитаем минимальный средний объём сырья в месторождениях региона, достаточный для его разработки.

In [16]:
min_product_region = BUDGET_FOR_REGION/BAR_PRICE/1000
print('Минимальный объем для окупаемости региона равен {:.2f} тыс. баррелей'.format(min_product_region))

Минимальный объем для окупаемости региона равен 2222.22 тыс. баррелей


Для того, чтобы окупить разработку месторождений в конкретном регионе, необходимо добыть из них не менее 2222.22 тыс. баррелей нефти.

Напишем функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели.

In [17]:
def profit(target, probabilities, count):
    '''
    Функция принимает в качестве аргументов: ответы, предсказания и размер выборки.
    Функция вычисляет прибыль по выборке с указанной размерностью.
    '''
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    result = BAR_PRICE * selected.sum()*1000 - COST_FOR_WELL*count
    return result

### Вывод

2 222.22 тыс. баррелей нефти - минимальный объем, который необходимо добыть в регионе, для того, чтобы окупить разработку месторождений. При этом на имеющийся бюджет мы можем разработать не более 200 месторождений в регионе. Для расчета прибыли по набору отобранных месторождений и предсказаний модели была написана функция.

# 4. Расчёт прибыли и рисков <a id = "4."> </a>

Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [18]:
bootstrap(target_valid_0, predictions_0)

Риск убытков: 0.00%
Средняя прибыль: 94.26
2.5%-квантиль: 88.98
97.5%-квантиль: 99.48


In [19]:
bootstrap(target_valid_1, predictions_1)

Риск убытков: 0.00%
Средняя прибыль: 95.15
2.5%-квантиль: 90.69
97.5%-квантиль: 99.32


In [20]:
bootstrap(target_valid_2, predictions_2)

Риск убытков: 0.00%
Средняя прибыль: 94.35
2.5%-квантиль: 88.71
97.5%-квантиль: 99.69


В результате мы видим, что во всех 3-х регионах отсутствует риск убытков. Это также подтверждается при сравнении средних значений объемов баррелей по регионам (92,69,94 тыс. бар.) и объема баррелей, необходимого чтобы выйти в 0 на скважине в регионе (11 тыс. бар.). Средняя прибыль во всех 3-х регионах примерно одинакова и раличается менее чем на 1 млрд. руб., но лидирует 2-й регион с результатом в 95.15 млрд. руб. Проанализировав доверительные интервалы, мы можем сделать вывод о то, что наиболее выгодным является для нас разработка месторождений во 2 регионе. Помимо того, что в доверительный интервал попадают значения с большей прибылью, так и обученная нами модель представляет собой идеальную модель предсказания. С большой долей уверенности можно сказать, что в этом регионе мы получим максимально возможную и предсказуемую прибыль и соответственно понесем меньше сопутствующих рисков.

### Вывод

Мы применили технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли. В результате анализа мы пришли в выводу о том, что наименее рисковым и выгодным для нас является 2 регион.

# Итог

Входящие данные были проанализированы. Были удалены избыточные признаки (колонка с id скважины). Для решения задачи была использована модель линейной регрессии. Для 3-х анализируемых регионов были обучены модели, предсказывающие объем сырья, который можно добыть на скважинах. Также был проанализирован бюджет и расходы и определена точка безубыточности в объемах сырья 2 222.22 тыс. бар. Для нахождения распределения прибыли была применена техника Bootstrap с 1000 выборок. В результате 2-й регион был определен как наиболее выгодный и наименее рисковый. Данный регион с высокой долей вероятности принесет от 90 до 100 млрд. руб. при бюджете в 10 млрд. Он будет предложен руководству для разработки новых месторождений.